<a href="https://colab.research.google.com/github/Matrix7043/Machine_learning101/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.metrics import confusion_matrix

In [3]:
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [7]:
np.random.seed(69)

In [8]:
def sigmoid(x):
    return 1/(1+np.exp(-1*x))

In [9]:
def loss(p,y):
    return -1*np.mean(y*np.log(p)+(1-y)*np.log((1-p)))

In [95]:
class LogisticRegression:

    def __init__(self, x, y, split, activation, loss, batch_size = None):
        self.x = x
        self.y = y
        ones = np.ones((self.x.shape[0],1))
        self.x = np.hstack((self.x,ones))

        split_ = int(len(x)*split)

        self.x = x[:split_]
        self.y = y[:split_]

        self.x_t = x_test[split_:]
        self.y_t = y_test[split_:]

        self.y_pred = np.zeros((self.y_t.shape[0], 1))

        self.w = np.random.rand(x.shape[1],1)
        self.w[-1][0] = 1
        self.batch_size = batch_size if batch_size else len(self.x)
        self.activation = activation
        self.loss = loss

    def forward(self, x_batch, y_batch):
        n = np.dot(x_batch, self.w)
        p = self.activation(n)
        c = self.loss(p, y_batch)
        return x_batch, y_batch, p

    def backward(self, x_batch, y_batch, p):
        return np.mean((p - y_batch) * x_batch, axis=0).reshape(-1, 1)

    def fit(self, epoch, learning_rate):
        for i in range(len(self.x)//self.batch_size):
            x_batch = self.x[self.batch_size*i: self.batch_size*(i+1)]
            y_batch = self.y[self.batch_size*i: self.batch_size*(i+1)]
            for _ in range(epoch):
                x_batch, y_batch, p = self.forward(x_batch, y_batch)
                dJ_dw = self.backward(x_batch, y_batch, p)
                self.w = self.w - learning_rate * dJ_dw

    def predict(self):
        xt = self.x_t
        yt = self.y_t
        self.y_pred = self.forward(xt, yt)[2]
        return self.y_pred

    def accuracy(self):
        max = [0, 0, 0]
        temp = [0, 0, 0]

        for t in range(100, 0, -1):
            temp[1] = t/100
            temp[2] = (self.y_pred > temp[1])
            yt = (self.y_t == temp[2])
            temp[0] = np.mean(yt)
            if (temp[0] > max[0]):
                max = temp[::]
        return max[0]


In [83]:
x2 = mnist["data"].head(1000).to_numpy()
y2 = mnist["target"].head(1000).to_numpy().reshape(-1,1).astype(int)
y_t = (y2==5).astype(int)
o = np.ones((x2.shape[0],1))
x_t = np.hstack((x2,o))

In [ ]:
log = LogisticRegression(x_t, y_t, 0.7, sigmoid, loss)

In [98]:
log.fit(1000, 0.001)

<ipython-input-9-45fe522252c0>:2: RuntimeWarning: invalid value encountered in multiply
  return -1*np.mean(y*np.log(p)+(1-y)*np.log((1-p)))
<ipython-input-8-2e8191186429>:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-1*x))


In [100]:
log.predict()

<ipython-input-8-2e8191186429>:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-1*x))
<ipython-input-9-45fe522252c0>:2: RuntimeWarning: invalid value encountered in multiply
  return -1*np.mean(y*np.log(p)+(1-y)*np.log((1-p)))


array([[2.85285792e-097],
       [0.00000000e+000],
       [0.00000000e+000],
       ...,
       [1.79828034e-162],
       [1.00000000e+000],
       [0.00000000e+000]])

In [102]:
print(log.accuracy())

0.9109677419354839
